# Marker gene heatmap of sandwich and dome culture data - Fig S1D

In [1]:
%matplotlib inline

import os
import scanpy as sc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

1.6.0
0.7.4


<div class="alert alert-info">
    
# Navigating this notebook
*** ***
**[Input/Output, Settings, and Upload Data](#in_out)**
0. [Directories](#step0)
1. [Plotting parameters](#plot_params)
2. [Load data](#load_data)
    <br>
    <br>

**[Heatmap](#heatmap_logic)**
3. [Parameters for marker gene visualization](#mg_params)
4. [Choose data and annotation levels (PLEASE READ)](#ann_level)
5. [Identify marker genes and order them by state](#id_mg_order)
6. [Gene filtering](#gene_filt)
7. [Prepare final heatmap - marker genes on any set of annotations](#prep_heatmap)
8. [Plot and save files](#plot_save)
    
</div>  

*** ***

<a id='in_out'></a>

## Input/Output & Upload Data

<a id='step0'></a>

### Directories for input and output data

In [5]:
# for AnnData objects from Scanpy:
# These AnnData objects contain output from the filtering and QC notebook
# Containing unnormalized counts matrices with low-count barcodes removed
h5ad_dir = '_h5ad_files'
os.makedirs(h5ad_dir, exist_ok=True)

# for plots from matplotlib, seaborn, helper functions plots:
plot_dir = '_plots_heatmap'
os.makedirs(plot_dir, exist_ok=True)

<a id='plot_params'></a>

### Plotting parameters

In [6]:
# Set up parameter settings for plots

# matplotlib params:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rc('font', size=12)
plt.rcParams['pdf.fonttype'] = 42

# scanpy settings:
sc.settings.verbosity = 2  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150)  # low dpi (dots per inch) yields small inline figure

#default for helper function plots from Klein lab:
hf.set_plot_defaults(fontsize=12)

# seaborn
sns.set(context='notebook', style='white', font_scale=1.8)

<a id='load_data'></a>

### Load data

In [7]:
adata = sc.read(f'{h5ad_dir}/adata_annotated_sand_dome.h5ad')

*** ***

<a id='heatmap_logic'></a>

## Code to make a heatmap of marker genes for single cell annotations / clusters

### Overall logic:

* We may wish to identify marker genes on one level of annotations (A1), but visualize them on another level of annotations (A2)
* We'll select genes if they have max expression in A1 annotations, ranked by max-to-second-max ratio
* We'll filter genes to have no more than N markers per annotation, *and* to have no less than max-to-second-max ratio R
* We will then plot them on A1 annotations
* We will then separately plot them on A2 annotations


<a id='mg_params'></a>

### Parameters for marker gene visualization

In [8]:
# Pseudocount value to use below for log-FC and max:2nd-max ratio
pseudocount = 1 # in units of CP10K

# Minimal acceptable max-2nd-max ratio:
max_2ndmax_ratio_thresh = 1.3

# Maximum number marker genes per state:
max_marker_genes = 150

# False discovery rate for p-value filter:
fdr = 0.05

# How to order the genes:
#gene_order_method = 'max_2ndmax_ratio'
gene_order_method = 'max_median_ratio'

<a id='ann_level'></a>

### Choose data and annotation levels (PLEASE READ)

* In the following, rename your scanpy frame to 'adata'
* In the following 'state' contains the annotation that is used for marker gene detection
* In the place clearly marked under 'final visualization', 'state' and 'treatment' are used for 

**If you wish to change the annotation column and final columns used for visualization, edit the appropriate text below**

In [9]:
# Give a clean name to the data:
adata.obs['state'] = adata.obs['state_coarse_grain']

# Here are the cluster annotations for which we'll identify marker genes:
print(adata.obs['state'].unique())

[Secretory, Enterocyte, Enteroendocrine, Stem, Tuft]
Categories (5, object): [Secretory, Enterocyte, Enteroendocrine, Stem, Tuft]


<a id='id_mg_order'></a>

### Identify marker genes and order them by state

*Currently, the ordering of states is not explicitly established - you get what Python gives you*

In [10]:
# Make a dataframe containing centroids:
centroids_df = pd.DataFrame(index=list(adata.raw.var_names))
for state in adata.obs['state'].unique():
    tmp = adata.raw[(adata.obs['state'] == state), : ].X.todense().sum(axis=0)
    centroids_df[state]= np.array(tmp).squeeze()

#normalize centroids to CP10K
for col in centroids_df.columns:
    centroids_df[col] = centroids_df[col]/centroids_df[col].sum()*1e4

<a id='gene_filt'></a>

### Gene filtering

#### Prepare information for gene filtering

In [11]:
# Rank-sum test on genes
# The p-values from these ranks will be used below as a filtering criteria
# We make no other usage of this function
if False:
    sc.tl.rank_genes_groups(adata, groupby='state', method='wilcoxon')

In [12]:
#create a dataframe with information used for filtering and normalization
centroids_info_df = pd.DataFrame()


# the cluster identity in which the gene is maximally expressed
centroids_info_df['max_state'] = centroids_df.idxmax(axis=1)

# the adjusted p-value for the gene in the max cluster 
centroids_info_df['pvals_adj'] = np.ones(np.shape(centroids_info_df['max_state']))
#for state in centroids_df.columns.unique():
    #centroids_info_df.loc[centroids_info_df['max_state'] == state, 'pvals_adj'] = \
        #adata.uns['rank_genes_groups']['pvals_adj'][state][centroids_info_df['max_state'] == state]
    #print(centroids_info_df.loc[centroids_info_df['max_state'] == state, 'pvals_adj'])
    #print(' ')

# the value of max expression, second max expression, and their ratio (pseudo-count corrected)
# NOTE: SECOND-MAX CALCULATION IS SLOW
centroids_info_df['max_CP10K'] = centroids_df.max(axis=1)
centroids_info_df['sec_max_CP10K'] = centroids_df.apply(lambda row: row.nlargest(2).values[-1], axis=1)
centroids_info_df['max_2nd_ratio'] = (pseudocount + centroids_info_df['max_CP10K']) / \
                                            (pseudocount + centroids_info_df['sec_max_CP10K'])

# Rank genes by their max-to-second-max ratio:
centroids_info_df['max_ratio_rank'] = centroids_info_df['max_2nd_ratio'].rank(ascending=False)

# for plotting - the cluster centroid median value per gene
centroids_info_df['med_CP10K'] = centroids_df.median(axis=1)

# max-median ratio:
centroids_info_df['max_med_ratio'] = (pseudocount+ centroids_info_df['max_CP10K']) / \
                                        (pseudocount+ centroids_info_df['med_CP10K'])


#### Apply filtering criteria

In [13]:
# CURRENTLY NOT DOING P-VALUE FILTERING, BECAUSE THE P-VALUES I HAVE FROM THE SCANPY PACKAGE
# SEEM TO MAKE NO SENSE

# Mask out genes that fail p-value test, by hiding their max-cluster assignment
# The masked-out max-state is stored in a new column, max_state_filt

centroids_info_df['max_state_filt'] = centroids_info_df['max_state']
#centroids_info_df.loc[ centroids_info_df['pvals_adj'] > fdr ,'max_state_filt'] = \
#    'non-significant'

In [14]:
# Apply remaining filter criteria to generate gene list
# Gene is max in the cluster
# Gene max-to-2nd-max is above threshold
# No more than max_marker_genes genes

marker_genes_dict = {}
marker_genes_all = []

for state in adata.obs['state'].unique():
    # List of genes that are maximal in the state:
    gl = centroids_info_df.index[(centroids_info_df['max_state_filt']==state)]
    
    # Get the top max_marker_genes from this list, ranked by max-to-second max
    # Using code from https://stackoverflow.com/questions/5807047/efficient-way-to-take-the-minimum-maximum-n-values-and-indices-from-a-matrix-usi
    # First get the lowest values for max_ratio_rank:
    n=len(centroids_info_df['max_ratio_rank'].loc[gl]) # number of genes significantly enriched and max in cluster
    if n > max_marker_genes:
        n=max_marker_genes        
    indices = np.argpartition(centroids_info_df['max_ratio_rank'].loc[gl].values, n-1)[:n]
        
        
    #print(centroids_info_df['max_ratio_rank'].loc[gl][indices])
    
    # Filter this list to eliminate genes that have less than threshold max-to-second max
    #print(centroids_info_df['max_2nd_ratio'].loc[gl][indices])
    indices = indices[centroids_info_df['max_2nd_ratio'].loc[gl][indices]>=max_2ndmax_ratio_thresh]
    #print(centroids_info_df['max_2nd_ratio'].loc[gl][indices])
    #print(' ')
    
    # Now we need the indices in order: (Default is gene_order_method='max_2ndmax_ratio')
    min_elements = centroids_info_df['max_ratio_rank'].loc[gl][indices]
    min_elements_order = np.argsort(min_elements)
    ordered_indices = indices[min_elements_order]
    #print(centroids_info_df['max_ratio_rank'].loc[gl][ordered_indices])
    #print(centroids_info_df['max_ratio_rank'].loc[gl][ordered_indices].index)        
    if gene_order_method=='max_median_ratio':
        #print('Ordering genes by max-median ratio (pseudocount-adjusted).')
        min_elements = centroids_info_df['max_med_ratio'].loc[gl][indices]
        min_elements_order = (np.argsort(min_elements)).iloc[::-1]
        ordered_indices = indices[min_elements_order]
    
    # Finally, here is the ordered list of gene names:
    gl = centroids_info_df['max_ratio_rank'].loc[gl][ordered_indices].index
    
    # Save this list:
    marker_genes_dict[state] = list(gl)
    marker_genes_all = marker_genes_all + list(gl)
    #print(gl)



<a id='prep_heatmap'></a>

### Prepare final heatmap - marker genes on any set of annotations

**EDIT THE FOLLOWING TO GET THE PLOT YOU WANT**

* *The code below currently makes a nested-columns dataframe that organizes the data*
    * Top level is state-annotation
    * Inner level is condition/treatment

In [17]:
# Make a dataframe containing centroids for plotting:
plt_centroids_df = pd.DataFrame(index=list(adata.raw.var_names))

for state in adata.obs['state'].unique():
    for condition in adata.obs['condition'].unique():
        tmp = adata.raw[(adata.obs['state'] == state) & \
                        (adata.obs['condition'] == condition), : ]\
                            .X.todense().sum(axis=0)
        plt_centroids_df[state, condition]= np.array(tmp).squeeze()

# Make hierarchical columns:
plt_centroids_df.columns = pd.MultiIndex.from_tuples(plt_centroids_df.columns)

#normalize centroids to CP10K
for col in plt_centroids_df.columns:
    plt_centroids_df[col] = plt_centroids_df[col]/plt_centroids_df[col].sum()*1e4



#### Save gene expression centroids

In [18]:
plt_centroids_df.to_excel(f'{plot_dir}/all_cells_centroids_sandwich_dome.xlsx')

In [19]:
# Now re-make the log fold-change data frame 
#   using same markers, but now for different sets of clusters
logFC_markers = (pseudocount + plt_centroids_df.loc[marker_genes_all]).divide( \
                pseudocount + centroids_info_df['med_CP10K'].loc[marker_genes_all],axis=0)

logFC_markers = np.log2(logFC_markers)

# Display:
logFC_markers

Secretory           Enterocyte           Enteroendocrine            \
             Dome  Sandwich       Dome  Sandwich            Dome  Sandwich   
defa30   5.855453  6.018702  -0.050642  0.016495        0.093473 -0.321338   
defa24   5.304220  5.105644  -0.917498 -1.091424        0.638501  0.220366   
lyz1     4.685358  3.824438  -0.136303 -0.445572        0.866685  0.341423   
spink4   4.410027  4.719428  -0.076234  0.083480        0.758533  1.006506   
defa29   4.155752  4.503432   0.065035  0.024894        0.021891 -0.213223   
...           ...       ...        ...       ...             ...       ...   
dyrk4    0.003576 -0.002531  -0.000761  0.000881       -0.002531 -0.002531   
alox5   -0.001645  0.005481   0.000012  0.005508       -0.004702 -0.004702   
rab25   -0.369916 -0.089643  -0.134638  0.191181       -0.202386  0.118138   
gm19817 -0.011522 -0.011522  -0.003870 -0.009474       -0.011522  0.038387   
rnf5    -0.165750  0.121532  -0.057937  0.075654       -0.025352  0.073082   

             Stem                Tuft            
             Dome  Sandwich      Dome  Sandwich  
defa30  -0.421781 -0.505114 -0.720269  2.499673  
defa24  -1.169352 -1.413878 -1.529305  1.548763  
lyz1    -0.116352 -0.723051 -0.425419  0.809377  
spink4  -0.138287 -0.706780 -0.666791 -0.525802  
defa29  -0.078861 -0.026162 -0.138262  0.343993  
...           ...       ...       ...       ...  
dyrk4   -0.002531 -0.002531  0.482894  0.208036  
alox5   -0.000796 -0.004702  0.452978  0.300643  
rab25   -0.044858  0.120146  0.388931  0.476244  
gm19817  0.004040 -0.011522  0.473902  0.199044  
rnf5    -0.165270  0.044194  0.205298  0.846471  

[398 rows x 10 columns]

<a id='plot_save'></a>

### Plot and save files

#### Plot and save the heatmap

In [22]:
# Make and format heatmap:
sns.set(font_scale=0.1)
sns.set(rc={'figure.figsize':(20,80)})
sns.heatmap(logFC_markers,cmap='coolwarm',vmin=-3,vmax=+3,xticklabels=1,yticklabels=1)

plt.tight_layout()

plt.savefig(f'{plot_dir}/heatmap_genes_by_state_and_condition_sandwich_dome_max150.eps')

#### Save the heatmap data

In [20]:
logFC_markers.to_excel(f'{plot_dir}/table_for_sandwich_dome_FC_figure_S1.xlsx')

#### Save marker gene list

In [21]:
with open('sandwich_dome_mg_list_mg150.txt', 'w') as f:
    for gene in list(logFC_markers.index):
        f.write("%s\n" % gene)